<a href="https://colab.research.google.com/github/pandi018/Covid_disease_classsification/blob/main/covid19_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pandiar","key":"406dbde3dd9cbe6f700e537baa340fdf"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d kaggle datasets download -d khoongweihao/covid19-xray-dataset-train-test-sets

usage: kaggle
       [-h]
       [-v]
       {competitions,c,datasets,d,kernels,k,config}
       ...
kaggle: error: unrecognized arguments: download


In [6]:
ls

kaggle.json  sample_data/


In [7]:
! kaggle datasets download -d khoongweihao/covid19-xray-dataset-train-test-sets

 95% 80.0M/84.3M [00:02<00:00, 33.0MB/s]
100% 84.3M/84.3M [00:02<00:00, 30.5MB/s]


In [8]:

! unzip /content/covid19-xray-dataset-train-test-sets.zip

Archive:  /content/covid19-xray-dataset-train-test-sets.zip
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0035-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0052-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0058-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0059-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0072-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0073-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0092-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0105-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0110-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0111-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0112-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL2-IM-0117-0001.jpeg  
  inflating: xray_dataset_covid19/test/NORMAL/NORMAL

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten 
from keras.models import Sequential 
from keras.preprocessing import image

#using generator i'm gonna do image augmentation

In [13]:
train_data=image.ImageDataGenerator(
    rescale=1/255,horizontal_flip=True,zoom_range=0.5,shear_range=0.2
) 
#i just done zooming and rescaleing using the image generator for more accuracy 

In [14]:
train_dataset=train_data.flow_from_directory(directory="/content/xray_dataset_covid19/train",target_size=(256,256),batch_size=1,class_mode="binary")

Found 148 images belonging to 2 classes.


In [15]:
train_dataset.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [17]:
test_data=image.ImageDataGenerator(
    rescale=1/255
) 
#here for test data i don't want any augmentations so i just rescale it  
test_dataset=test_data.flow_from_directory(directory="/content/xray_dataset_covid19/test",target_size=(256,256),batch_size=10,class_mode="binary")

Found 40 images belonging to 2 classes.


In [18]:
test_dataset.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

#now i'm gonna create the CNN model

In [19]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(256,256,3)))

In [20]:
#there 256,256 represent the value and 3 detnotes the RCB color 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))
#dropout is used to avoid overfitting 

In [21]:
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.20))

In [22]:
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

In [23]:
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Dropout(rate=0.40))

In [24]:
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.30)) 

In [25]:
model.add(Dense(1,activation='tanh'))

In [26]:
model.compile(
optimizer="adam", 
loss="binary_crossentropy",
metrics= ['accuracy'])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                    

In [28]:
histroy=model.fit_generator(train_dataset,epochs=9,steps_per_epoch=6,validation_steps=2,
                    validation_data=test_dataset)

<ipython-input-28-99ecadd99ef4>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  histroy=model.fit_generator(train_dataset,epochs=9,steps_per_epoch=6,validation_steps=2,


Epoch 1/9
6/6 [==============================] - 15s 3s/step - loss: 2.5853 - accuracy: 0.8333 - val_loss: 9.2550 - val_accuracy: 0.4000
Epoch 2/9
6/6 [==============================] - 6s 1s/step - loss: 10.2833 - accuracy: 0.3333 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 3/9
6/6 [==============================] - 6s 1s/step - loss: 2.5708 - accuracy: 0.8333 - val_loss: 6.1700 - val_accuracy: 0.6000
Epoch 4/9
6/6 [==============================] - 8s 1s/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 7.7125 - val_accuracy: 0.5000
Epoch 5/9
6/6 [==============================] - 7s 1s/step - loss: 7.7125 - accuracy: 0.5000 - val_loss: 4.6275 - val_accuracy: 0.7000
Epoch 6/9
6/6 [==============================] - 7s 1s/step - loss: 10.2833 - accuracy: 0.3333 - val_loss: 8.4837 - val_accuracy: 0.4500
Epoch 7/9
6/6 [==============================] - 7s 1s/step - loss: 5.1416 - accuracy: 0.6667 - val_loss: 8.4837 - val_accuracy: 0.4500
Epoch 8/9
6/6 [==============================